In [ ]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [110]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [92]:
from getpass import getpass

OPENAI_API_KEY = getpass()

········


In [93]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Creating a CSV agent to interact with and analyze data

In [94]:
agent = create_csv_agent(OpenAI(temperature=0),
        '/Users/sara/Downloads/my_whoop_data_2023_04_29/physiological_cycles.csv')

In [11]:
ans = agent.run("what are the columns in this CSV file?")

Index(['Cycle start time', 'Cycle end time', 'Cycle timezone',
       'Recovery score %', 'Resting heart rate (bpm)',
       'Heart rate variability (ms)', 'Skin temp (celsius)', 'Blood oxygen %',
       'Day Strain', 'Energy burned (cal)', 'Max HR (bpm)', 'Average HR (bpm)',
       'Sleep onset', 'Wake onset', 'Sleep performance %',
       'Respiratory rate (rpm)', 'Asleep duration (min)',
       'In bed duration (min)', 'Light sleep duration (min)',
       'Deep (SWS) duration (min)', 'REM duration (min)',
       'Awake duration (min)', 'Sleep need (min)', 'Sleep debt (min)',
       'Sleep efficiency %'],
      dtype='object')


### Creating a Coach LLM
We create a prompt asking the coach for data analysis that would help it advise us on how to increase HRV.

In [130]:
coach_llm = OpenAI(temperature=0.9)

In [131]:
text = "You are a fitness coach. You have daily data on my" + ans + " for the past 2 weeks. \
You want to give me feedback on how to increase my heart rate variability. \
You can ask a data analyst to do computation on this data for you. \
Give me a list of 4 data analysis questions you would ask to tell me between more exercise, \
more REM sleep, and Max HR, which one has the highest effect on heart rate variability? \
Data analysis can only be in the form of numbers, not charts, or tables.\
Respond to this question in the following form: \
1. \n\
2. \n\
3. \n\
4. \n " 


In [136]:
plan = coach_llm(text).split("\n")[1:]

In [137]:
plan

['1. What is the correlation between Max HR and Heart Rate Variability? ',
 '2. What is the average change in Heart Rate Variability when individuals exercise? ',
 '3. What is the average change in Heart Rate Variability when individuals have improved REM sleep? ',
 '4. What is the average change in Heart Rate Variability when individuals increase their average HR?']

This function creates a feedback loop where the coach can check whether the CSV agent has returned the data it requested.

In [138]:
def confirmResult(i, analysis, plan): 
    prompt = "The response to " + plan[i] + " is the sentence in the brackets: \
    {" + analysis + "}. Is this answer relevant? Answer with `0` if it's not relevant and `1` if it is."
    ans = int(coach_llm(prompt).replace("\n", ""))
    if ans == 0:
        new_prompt = "Reword " + plan[i] + " asking for numerical insights."
        plan[i] = coach_llm(new_prompt)
        print(plan[i])
        return False
    return True
    

### The CSV agent executing the plan 
The agent answers each prompt while getting feedback from the coach. 

In [ ]:
answers = []
for i in range(0,len(plan)): 
    print(plan[i])
    curr_analysis = agent.run(plan[i]) 
    print(curr_analysis)
    if confirmResult(i, curr_analysis, plan) == False: 
        curr_analysis = agent.run(plan[i]) 
        print(curr_analysis)
    answers.append(agent.run(plan[i]))

1. What is the correlation between Max HR and Heart Rate Variability? 
The correlation between Max HR and Heart Rate Variability is 0.28.
1


In [140]:
final_prompt = "You are a health specialist. I will provide you with 4 questions\
related to my health statistics and answers to them. Given this data, \
tell me between more exercise, \
more REM sleep, and Max HR, which one I should increase to improve my HRV and why? Use the numbers provided in the\
in the answer in your reasoning as well as your knowledge on this.\
Questions: \n" + "".join(plan) + "\n Answers: \n" + "".join(answers)  

In [141]:
coach_llm(final_prompt)

'\n\nGiven this data, I would recommend increasing your Max HR to improve your HRV. This is because increasing your Max HR has the highest correlation with HRV, which indicates the two are strongly correlated. Additionally, the average change in Heart Rate Variability when individuals increase their average HR is -0.5686274509803921, which is lower than the average change in Heart Rate Variability when individuals increase their Max HR. Therefore, increasing your Max HR should result in the most significant improvement to your HRV.'